In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
# Load dataset
file_path = "Bondora_raw.csv"
df = pd.read_csv(file_path)

# Display basic info
print(df.info())
print(df.head())

C:\Users\sanch\AppData\Local\Temp\ipykernel_19592\3228195612.py:3: DtypeWarning: Columns (34,37,38,80,82,83,89,90) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134529 entries, 0 to 134528
Columns: 112 entries, ReportAsOfEOD to ActiveLateLastPaymentCategory
dtypes: bool(3), float64(56), int64(14), object(39)
memory usage: 112.3+ MB
None
  ReportAsOfEOD                                LoanId  LoanNumber  \
0    2020-01-27  F0660C80-83F3-4A97-8DA0-9C250112D6EC         659   
1    2020-01-27  978BB85B-1C69-4D51-8447-9C240104A3A2         654   
2    2020-01-27  EA44027E-7FA7-4BB2-846D-9C1F013C8A22         641   
3    2020-01-27  CE67AD25-2951-4BEE-96BD-9C2700C61EF4         668   
4    2020-01-27  9408BF8C-B159-4D6A-9D61-9C2400A986E3         652   

           ListedOnUTC     BiddingStartedOn  BidsPortfolioManager  BidsApi  \
0  2009-06-11 16:40:39  2009-06-11 16:40:39                     0        0   
1  2009-06-10 15:48:57  2009-06-10 15:48:57                     0        0   
2  2009-06-05 19:12:29  2009-06-05 19:12:29                     0        0   
3  2009-06-13 12:01:20  2009-06-13 12:01:20  

In [3]:
# Identify missing values
missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values)

Missing Values:
 ReportAsOfEOD                              0
LoanId                                     0
LoanNumber                                 0
ListedOnUTC                                0
BiddingStartedOn                           0
                                       ...  
NrOfScheduledPayments                  36741
ReScheduledOn                          94207
PrincipalDebtServicingCost             75400
InterestAndPenaltyDebtServicingCost    75400
ActiveLateLastPaymentCategory          82279
Length: 112, dtype: int64


In [4]:
# Drop columns with more than 50% missing values
threshold = len(df) * 0.5
df = df.dropna(thresh=threshold, axis=1)

In [5]:
# Fill missing values
for col in df.select_dtypes(include=['number']):
    df[col] = df[col].fillna(df[col].median())

for col in df.select_dtypes(include=['object']):
    df[col] = df[col].fillna("Unknown")

In [6]:
# Remove duplicate rows
df = df.drop_duplicates()

In [7]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [8]:
# Convert date columns
date_cols = ['reportasofeod', 'listedonutc', 'biddingstartedon', 'loanapplicationstarteddate',
    'loandate', 'firstpaymentdate', 'maturitydate_original', 'maturitydate_last',
    'lastpaymenton', 'stageactivesince']
for col in date_cols:
    df[col] = pd.to_datetime(df[col],errors='coerce')

C:\Users\sanch\AppData\Local\Temp\ipykernel_19592\3863812532.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col],errors='coerce')


In [9]:
# Check unique values in 'status'
print(df['status'].value_counts())

status
Current    57135
Late       45772
Repaid     31622
Name: count, dtype: int64


In [10]:
# Map 'status' column to binary target
default_statuses = ["Current", "Late", "Repaid"]  # Update based on actual values
df['target'] = df['status'].apply(lambda x: 1 if x == "Late" else 0)

# Verify distribution
print(df['target'].value_counts(normalize=True))

target
0    0.659761
1    0.340239
Name: proportion, dtype: float64


In [11]:
# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Label Encoding for binary categorical columns
le = LabelEncoder()
for col in categorical_cols:
    if df[col].nunique() == 2:
        df[col] = le.fit_transform(df[col])

In [12]:
# Cap outliers to 1st and 99th percentile
for col in df.select_dtypes(include=['number']):
    lower = df[col].quantile(0.01)
    upper = df[col].quantile(0.99)
    df[col] = df[col].clip(lower, upper)

In [13]:
df.to_csv('Bondora_processed.csv', index=False)
print("Processed dataset saved.")

Processed dataset saved.
